# Hands-On Exercise 9.1: Converting XML to JSON

## Objective

Write a command line tool to convert an XML document containing football match details into JSON.
    
This is a realistic problem as the XML format is less-well supported that it used to be, while the tool development around JSON is very active.

Close the previous solution if you have not already done so.

#### Review the XML document

Open the XML document in `C:\Course\510D\Data\epl_2020_2021.xml`. You can use Visual Studio, Visual Studio Code, Notepad, etc. It's just a text file.

Review the contents. It's a list of the (380) matches played in the 2020/2021 season of the English Premier League. Each match is describe by a `<Match />` element.

Here's the first one.

```XML
<Match id="0">
    <Kickoff>2020-09-12T12:30:00+01:00</Kickoff>
    <HomeTeam>Fulham</HomeTeam>
    <AwayTeam>Arsenal</AwayTeam>
    <Score>
        <FullTime>
            <Home>0</Home>
            <Away>3</Away>
        </FullTime>
        <HalfTime>
            <Home>0</Home>
            <Away>1</Away>
        </HalfTime>
    </Score>
    <Prices>
        <Home>6.16</Home>
        <Draw>4.51</Draw>
        <Away>1.56</Away>
    </Prices>
</Match>
```

This represents the game on 12-Sep-2020 where Fulham were at home to Arsenal. Arsenal were leading 0-1 at half-time and eventually won the game with a score line of 0-3. The betting prices on the day were 6.16 for a Fulham win, 4.51 for a draw and 1.56 for an Arsenal win.

The `id` attribute is a unique ID.

#### Create a new console application targeting .NET 6.0.

Create a new console app project/solution. Name it `XmlToJson` with the location `C:\Course\510D\Exercises\`.

<font color="red">**When asked what framework to use, choose NET _5.0_.**</font>

This will ensure we get a `Program` class.

When the solution has been created, right-click on the `XmlToJson` _project_ in Solution Explorer and open the **Properties** editor.

In the **Application | General** tab change the **Target Framework** to .NET 6.0. In the **Build | General** tab, enable **Nullable** (non-nullable reference types).

We now have a .NET 6 application _and_ a full `Program` class.

#### Read the XML document path from the command line.

To make this tool realistic, the path to the XML document will be taken from the command line. Hard-coded paths are a pain for users.

Parameters passed on the command line are available as the `args` parameter passed to the `Program` class' `Main` method.

When the application is run in Visual Studio (i.e. during development) this command line paramater will need to be passed to our application. We can specify this using the project's property editor.

Go to the property editor's **Debug | General** tab. Click on the **Open debug launch profiles UI** link. In the **Command line arguments** text box, enter

```
C:\Course\510D\Data\epl_2020_2021.xml
```

Confirm that only one argument was specified via the command line. If this _isn't_ the case, display a message to the user and exit the application.

You can use `Application.Exit(1);` to exit the application with error code 1.

#### Answer...

In [ ]:
if (args.Length != 1)
{
    Console.WriteLine("Usage: <XML document path>");

    Environment.Exit(1);
}

#### Create a match model class.

The XML document will be deserialized to list of .NET objects---a list of `Match` objects.
    
Create a `Match` object with properties for all the data provided by the XML document.

It will look something like

```C#
public class Match
{
    public int Id { get; set; }
    public DateTime Kickoff { get; set; }
    public string HomeTeam { get; set; } = string.Empty;
    public string AwayTeam { get; set; } = string.Empty;
    public Score Score { get; set; } = new();
    public Score HalfTimeScore { get; set; } = new();
    public Prices Prices { get; set; } = new();
}
```

The nested structure of the XML document has been "flattened". This isn't necessary, but it makes the code simpler. 

It's also realistic as it's an easier format to use if you want to injest it into tablular analysis tools, such as spreadsheets and databases.

#### Write a function to import a list of `Matches` from the XML document.

Scaffold a `private static` `ImportMatchesFromXml` that takes the path to the XML document and returns `IEnumerable<Match>`.

#### Answer...

In [ ]:
private static IEnumerable<Match> ImportMatchesFromXml(string documentPath)
{
}

In `ImportMatchesFromXml`, read the XML document using XML to LINQ. The code to do this will look like

```C#
XDocument document = XDocument.Load(documentPath);
```

Create an empty `matches` collection to hold the `Match` objects that will be generated. It will look like this
    
```C#
ICollection<Match> matches = new List<Match>();
```

The top-level `<Match />` elements are in `document.Root`.

A list of the match `XElement`s can be obtained using `document.Root!.Elements()`. The use of the null-forgiving operator (`!`) is justified here as XML documents usually adhere to a strict schema definition---and are pre-validated.

Scaffold a `foreach` loop that iterates over the match `XElement`s.

#### Answer...

In [ ]:
foreach (XElement matchElement in document.Root!.Elements())
{
}

Inside the `foreach` loop create a new `Match` object. Assign its `Id` property using the `id` attribute from the `<Match />` element.

The code will look like

```C#
var match = new Match
{
    Id = (int)matchElement.Attribute("id")!
}
```

Note the use of the null-forgiving (`!`) operator again.

Initialize the rest of the properties in the same way. The complete statement should look similar to

```C#
var match = new Match
{
    Id = (int)matchElement.Attribute("id")!,
    Kickoff = DateTime.Parse((string)matchElement.Element("Kickoff")!),
    HomeTeam = (string)matchElement.Element("HomeTeam")!,
    AwayTeam = (string)matchElement.Element("AwayTeam")!,
    HomeTeamGoals = (int)matchElement.Element("Score")!.Element("FullTime")!.Element("Home")!,
    AwayTeamGoals = (int)matchElement.Element("Score")!.Element("FullTime")!.Element("Away")!,
    HalfTimeHomeTeamGoals = (int)matchElement.Element("Score")!.Element("HalfTime")!.Element("Home")!,
    HalfTimeAwayTeamGoals = (int)matchElement.Element("Score")!.Element("HalfTime")!.Element("Away")!,
    HomeWinPrice = (decimal)matchElement.Element("Prices")!.Element("Home")!,
    DrawPrice = (decimal)matchElement.Element("Prices")!.Element("Draw")!,
    AwayWinPrice = (decimal)matchElement.Element("Prices")!.Element("Away")!,
};
```

Add the new `match` to the list of `matches`.

#### Answer...

In [ ]:
matches.Add(match);

Return the list of `matches` at the end of the `ImportMatchesFromXml` function.

Do a build to check all is well.

In the `Main` function, call `ImportMatchesFromXml` with the first command line argument and store it's return value in `matches`.

#### Answer...

In [ ]:
IEnumerable<Match> matches = ImportMatchesFromXml(args[0]).ToList();

Output the length of the matches list using `Console.WriteLine($"There are {matches.Count()} matches.");`.

How many matches are there in the document?

#### Answer...

380 (20 teams playing each other---twice).

#### Output the list of matches as JSON.

Scaffold an `ExportMatchesToJson` function that takes the `IEnumerable<Match>` containing the matches.

It should like like this

```C#
private static void ExportMatchesToJson(IEnumerable<Match> matches)
{
}
```

Most JSON documents use camel-cased properties (e.g. `homeTeam`) and, as this document will be reviewed, it should have a "pretty" layout.

Inside `ExportMatchesToJson` create a `JsonSerializerOptions` object that specifies camel-cased property naming and indentation. The code will look like

```C#
var serializerOptions = new JsonSerializerOptions
{
    PropertyNamingPolicy = JsonNamingPolicy.CamelCase,
    WriteIndented = true
};
```

Using the `serializerOptions`, serialize the `matches` to a JSON string (`json`).

#### Answer...

In [ ]:
string json = JsonSerializer.Serialize(matches, serializerOptions);

Write the JSON string out to the console.

#### Answer...

In [ ]:
Console.Write(json);

Another option would be to write the JSON to a file using `File.WriteAllText(json);`. However, writing to the console makes it easy to confirm the output looks sensible, and we can easily redirect the output to a file using `>` in the terminal.

Return to the `Main` method and remove the `Console.WriteLine` statement that displays the number of matches.

Call `ExportMatchesToJson`, passing it the `matches` returned from `ImportMatchesFromXml`.

The code should look like

```C#
ExportMatchesToJson(matches);
```

Run and test the application.

## Congratulations! You have successfully completed the exercise. Continue to the bonus if you have more time.

# Bonus (Optional)

The JSON document the application generates uses a "flat" structure (no nested objects). Create sub-objects to store scores and prices.

#### Create new model classes.

Create a `Score` class with `HomeTeamGoals` and `AwayTeamGoals` properties.

#### Answer...

In [ ]:
public class Score
{
    public int HomeTeamGoals { get; set; }
    public int AwayTeamGoals { get; set; }
}

Create a `Prices` class with `HomeWin`, `Draw` and `AwayWin` properties (all `decimal`).

#### Answer...

In [ ]:
public class Prices
{
    public decimal HomeWin { get; set; }
    public decimal Draw { get; set; }
    public decimal AwayWin { get; set; }
}

Refactor the `Match` class to use these new classes.

The new `Match` class should look like

```C#
public class Match
{
    public int Id { get; set; }
    public DateTime Kickoff { get; set; }
    public string HomeTeam { get; set; } = string.Empty;
    public string AwayTeam { get; set; } = string.Empty;
    public Score Score { get; set; } = new();
    public Score HalfTimeScore { get; set; } = new();
    public Prices Prices { get; set; } = new();
}
```

Go to the `ImportMatchesFromXml` function. When creating the `match`, serialize the goals and prices into the new properties.
    
The new statement should look like

```C#
var match = new Match
{
    Id = (int)matchElement.Attribute("id")!,
    Kickoff = DateTime.Parse((string)matchElement.Element("Kickoff")!),
    HomeTeam = (string)matchElement.Element("HomeTeam")!,
    AwayTeam = (string)matchElement.Element("AwayTeam")!,
    Score = new Score
    {
        HomeTeamGoals = (int)matchElement.Element("Score")!.Element("FullTime")!.Element("Home")!,
        AwayTeamGoals = (int)matchElement.Element("Score")!.Element("FullTime")!.Element("Away")!
    },
    HalfTimeScore = new Score
    {
        HomeTeamGoals = (int)matchElement.Element("Score")!.Element("HalfTime")!.Element("Home")!,
        AwayTeamGoals = (int)matchElement.Element("Score")!.Element("HalfTime")!.Element("Away")!,
    },
    Prices = new Prices
    {
        HomeWin = (decimal)matchElement.Element("Prices")!.Element("Home")!,
        Draw = (decimal)matchElement.Element("Prices")!.Element("Draw")!,
        AwayWin = (decimal)matchElement.Element("Prices")!.Element("Away")!,
    }
};
```

Run and test the application. Review some of the match objects in the JSON to see how the goals and prices are now represented.
    
The first match should look like

```JSON
{
  "id": 0,
  "kickoff": "2020-09-12T12:30:00+01:00",
  "homeTeam": "Fulham",
  "awayTeam": "Arsenal",
  "score": {
    "homeTeamGoals": 0,
    "awayTeamGoals": 3
  },
  "halfTimeScore": {
    "homeTeamGoals": 0,
    "awayTeamGoals": 1
  },
  "prices": {
    "homeWin": 6.16,
    "draw": 4.51,
    "awayWin": 1.56
  }
}
```

Note that it wasn't necessary to touch the JSON serialization code. It was able to reflect on the new definitions and serialize accordingly.

## Congratulations! You have completed the bonus.